In [1]:
import petsc4py.PETSc as PETSc
import underworld3 as uw
import numpy as np

In [2]:
mesh = uw.FeMesh((6,6), simplex=False)

In [3]:
dim = mesh.plex.getDimension()

In [4]:
# Get Element Count
eStart, eEnd = mesh.plex.getHeightStratum(0)
elCount = eEnd - eStart

# Build Velocity ad Solution vectors

In [5]:
dim = mesh.plex.getDimension()

In [6]:
ds = mesh.plex.getDS()

In [7]:
# Get Communicator from DM
comm = mesh.plex.comm
# Create Default FE
fe_vel = PETSc.FE().createDefault(dim=mesh.dim, nc=2, isSimplex=mesh.isSimplex, qorder=-1, comm=comm)
# Set Name
fe_vel.setName("velocity")
# Get Quadrature
fe_vel_quad = fe_vel.getQuadrature()
# Get Face Quadrature
fe_vel_face_quad = fe_vel.getFaceQuadrature()
# Set Discretisation
ds.setDiscretisation(0, fe_vel)
fe_vel.destroy()

In [8]:
# Create Default FE
fe_pres = PETSc.FE().createDefault(dim=mesh.dim, nc=1, isSimplex=mesh.isSimplex, qorder=-1, comm=comm)
# Set Name
fe_pres.setName("pressure")
# Set Quadrature
fe_pres.setQuadrature(fe_vel_quad)
# Set Face Quadrature
fe_pres.setFaceQuadrature(fe_vel_face_quad)
# Set Discretisation
ds.setDiscretisation(1, fe_pres)
fe_pres.destroy()

In [9]:
swarm = PETSc.DMSwarm().create()

In [10]:
swarm.setDimension(dim)

In [11]:
swarm.setType(1)

In [12]:
swarm.setCellDM(mesh.plex)

In [13]:
swarm.registerField("eta0", 1, dtype=PETSc.RealType)

In [14]:
swarm.registerField("rank", 1, dtype=PETSc.IntType)

In [15]:
swarm.finalizeFieldRegister()

In [16]:
ppcell = 2

In [17]:
swarm.setLocalSizes(elCount*ppcell, 0)

In [18]:
swarm.insertPointUsingCellDM(1, ppcell)

In [19]:
swarm.getSize()

144

### Initialize Swarm Variables

In [20]:
coords = swarm.getField("DMSwarmPIC_coor")

In [21]:
eta = swarm.getField("eta0")
rank0 = swarm.getField("rank")

In [22]:
lsize = swarm.getLocalSize()

In [23]:
x = coords[0::2]
y = coords[1::2]

In [24]:
rank0[:] = 0

In [25]:
import numpy as np
eta[:] = np.cos(4*np.pi*x) * np.cos(2*np.pi*y)

In [26]:
eta

array([ 0.88160516,  0.6125759 , -0.07893927, -0.05485028, -0.80266589,
       -0.55772562, -0.80266589, -0.55772562, -0.07893927, -0.05485028,
        0.88160516,  0.6125759 ,  0.88160516,  0.6125759 , -0.07893927,
       -0.05485028, -0.80266589, -0.55772562, -0.80266589, -0.55772562,
       -0.07893927, -0.05485028,  0.88160516,  0.6125759 ,  0.26902926,
       -0.26902926, -0.02408898,  0.02408898, -0.24494027,  0.24494027,
       -0.24494027,  0.24494027, -0.02408898,  0.02408898,  0.26902926,
       -0.26902926,  0.26902926, -0.26902926, -0.02408898,  0.02408898,
       -0.24494027,  0.24494027, -0.24494027,  0.24494027, -0.02408898,
        0.02408898,  0.26902926, -0.26902926, -0.6125759 , -0.88160516,
        0.05485028,  0.07893927,  0.55772562,  0.80266589,  0.55772562,
        0.80266589,  0.05485028,  0.07893927, -0.6125759 , -0.88160516,
       -0.6125759 , -0.88160516,  0.05485028,  0.07893927,  0.55772562,
        0.80266589,  0.55772562,  0.80266589,  0.05485028,  0.07

In [27]:
swarm.restoreField("DMSwarmPIC_coor")
swarm.restoreField("eta0")
swarm.restoreField("rank")

In [28]:
#swarm.viewXDMF("swarm.xmf")

In [29]:
#swarm.viewFieldsXDMF("swarm2.xmf", ["eta0", "rank"])

In [30]:
swarm.view()